In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
#using model.save_weights() to save the weights of the model in HDF5 format
# model.save_weights("/content/gdrive/My Drive/weights.h5")

Mounted at /content/gdrive


In [3]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops

from datasets import load_dataset
from random import randrange

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM

from trl import SFTTrainer

from huggingface_hub import login

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.1 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [ ]:
!pip install accelerate

import accelerate

In [7]:
model_name = "Salesforce/codegen-350M-mono"
dataset_name = "iamtarun/python_code_instructions_18k_alpaca"

In [8]:
from huggingface_hub import notebook_login
# Log in to HF Hub
notebook_login()

In [9]:
from datasets import load_dataset_builder
from datasets import load_dataset
from random import randrange
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,TrainingArguments, Trainer
import numpy as np

code_dataset = load_dataset('iamtarun/python_code_instructions_18k_alpaca', split='train[:5000]')
model_name = "Salesforce/codegen-350M-mono"


model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
code_dataset = code_dataset.train_test_split(test_size=0.2)
code_dataset['train'][0]

{'instruction': 'Write a Python function to classify data points as low, medium and high according to the given threshold values.',
 'input': 'threshold = [4, 6]',
 'output': "def classify(value, threshold):\n    if value < threshold[0]:\n        return 'low'\n    elif threshold[0] <= value < threshold[1]:\n        return 'medium'\n    else:\n        return 'high'",
 'prompt': "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWrite a Python function to classify data points as low, medium and high according to the given threshold values.\n\n### Input:\nthreshold = [4, 6]\n\n### Output:\ndef classify(value, threshold):\n    if value < threshold[0]:\n        return 'low'\n    elif threshold[0] <= value < threshold[1]:\n        return 'medium'\n    else:\n        return 'high'"}

In [11]:
code_dataset = code_dataset.flatten()
code_dataset['train'][0]


{'instruction': 'Write a Python function to classify data points as low, medium and high according to the given threshold values.',
 'input': 'threshold = [4, 6]',
 'output': "def classify(value, threshold):\n    if value < threshold[0]:\n        return 'low'\n    elif threshold[0] <= value < threshold[1]:\n        return 'medium'\n    else:\n        return 'high'",
 'prompt': "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWrite a Python function to classify data points as low, medium and high according to the given threshold values.\n\n### Input:\nthreshold = [4, 6]\n\n### Output:\ndef classify(value, threshold):\n    if value < threshold[0]:\n        return 'low'\n    elif threshold[0] <= value < threshold[1]:\n        return 'medium'\n    else:\n        return 'high'"}

In [12]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["prompt"]])

In [13]:

tokenized_code_dataset = code_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=code_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2122 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5271 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4658 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (11388 > 2048). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (6735 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4305 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5956 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (12474 > 2048). Running this sequence through the model will result in indexing errors


In [14]:

block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [15]:
lm_code_dataset = tokenized_code_dataset.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [18]:
lm_code_dataset['train'][0]

{'input_ids': [33,
  304,
  300,
  267,
  266,
  50285,
  72,
  264,
  50285,
  64,
  299,
  50285,
  72,
  299,
  264,
  256,
  374,
  334,
  269,
  256,
  1312,
  267,
  299,
  50285,
  83,
  289,
  257,
  256,
  50285,
  67,
  304,
  264,
  269,
  374,
  1312,
  275,
  304,
  264,
  50285,
  64,
  50285,
  83,
  257,
  264,
  479,
  764,
  50285,
  54,
  374,
  1312,
  256,
  304,
  50285,
  64,
  50285,
  81,
  304,
  264,
  279,
  267,
  299,
  264,
  304,
  50285,
  83,
  289,
  257,
  256,
  50285,
  64,
  279,
  279,
  374,
  267,
  279,
  374,
  1312,
  257,
  256,
  304,
  300,
  331,
  50285,
  66,
  267,
  285,
  279,
  300,
  304,
  256,
  304,
  264,
  50285,
  83,
  289,
  304,
  50285,
  81,
  304,
  10662,
  334,
  304,
  264,
  256,
  764,
  220,
  198,
  220,
  198,
  1303,
  1303,
  1303,
  50285,
  40,
  299,
  264,
  256,
  374,
  334,
  269,
  256,
  1312,
  267,
  299,
  1058,
  220,
  198,
  370],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 

In [16]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [19]:

training_args = TrainingArguments(
    output_dir="./colab_code_model_UT",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_code_dataset["train"],
    eval_dataset=lm_code_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.654000,0.722431


Epoch,Training Loss,Validation Loss
1,0.654000,0.722431
2,0.495300,0.705233
3,0.367500,0.775352


TrainOutput(global_step=8403, training_loss=0.5535196188332339, metrics={'train_runtime': 7545.1145, 'train_samples_per_second': 8.908, 'train_steps_per_second': 1.114, 'total_flos': 1.570746270744576e+16, 'train_loss': 0.5535196188332339, 'epoch': 3.0})

In [20]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 2.17


In [36]:
prompt = "Write a Python function to classify data points as low, medium and high according to the given threshold values"

# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("01GangaPutraBheeshma/colab_code_model_UT")
# model = AutoModelForCausalLM.from_pretrained("01GangaPutraBheeshma/colab_code_model_UT")


tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer(prompt, return_tensors="pt").attention_mask

trained_model = AutoModelForCausalLM.from_pretrained("01GangaPutraBheeshma/colab_code_model_UT")
outputs_1 = trained_model.generate(inputs, max_new_tokens=500, do_sample=True, top_k=50, top_p=0.95)

# outputs_2 = trained_model.generate(**inputs, num_beams=4, do_sample=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [37]:
tokenizer.batch_decode(outputs_1, skip_special_tokens=True)

['"""""""""""""""""""" d e s t e r   o f   p r e d i c t i on\'t   c i t i v a t i o n   v a l i d a t i o n " \n                 " " " \n                 \n                 s e l f. n e w _ t o k e n _ i d   =   N o n e \n \n                 d e f   p a r e n t _ c o n t r o l l e r _ t o k e n ( s e l f ) : \n                         #   p a r e n t   t o k e n   t o   s t o r e   s i z e   o f   t h e   c o r o n a v i r u s   o f   t h e   t o k e n \n                         s e l f. n e w _ t o k e n _ i d   =   N o n e \n \n                 d e f   p a r e n t _ c o n t e x t _ t o k e n ( s e l f ) : \n                         p r i n t   " P a r e n t   c o n t e x t   t o   t o k e n   { 0 } ". f o r m a t ( s e l f. c u r r e n t _ t o k e n _ i d ) \n                         s e l f. n e w _ t o k e n _ i d   =   N o n e \n \n                         #   c h e c k   w h e n   i n p u t   f o r   t h e   s i z e   o f   t h e   c o r o n a v i r u s   o f   t h e   t o k e n

In [ ]:
tokenizer.batch_decode(outputs_2, skip_special_tokens=True)


In [ ]:
trainer.push_to_hub()
